<a href="https://colab.research.google.com/github/ignacioreyna/caba/blob/master/Urban_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!mkdir downloads
!pip install pyshp
!pip install geopandas

In [0]:
import urllib.request
import zipfile
import os
import pandas as pd
import shapefile
import numpy as np
import matplotlib.pyplot as plt
import geopandas
import seaborn as sns
import matplotlib as mpl

mpl.rcParams['figure.dpi']= 400
sns.set(style='white', palette='pastel', color_codes=True)
sns.mpl.rc('figure', figsize=(15,9))

In [0]:
tejidos = [
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Agronomia_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Almagro_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Balvanera_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Barracas_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Belgrano_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Boca_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Boedo_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Caballito_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Chacarita_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Coghlan_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Colegiales_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Constitucion_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Floresta_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Flores_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Liniers_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Mataderos_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Monserrat_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Monte_Castro_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Nueva_Pompeya_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Nunez_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Palermo_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Parque_Avellaneda_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Parque_Chacabuco_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Parque_Chas_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Parque_Patricios_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Paternal_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Puerto_Madero_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Recoleta_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Retiro_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Saavedra_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/San_Cristobal_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/San_Nicolas_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/San_Telmo_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Velez_Sarsfield_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Versalles_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Villa_Crespo_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Villa_Del_Parque_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Villa_Devoto_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Villa_Gral_Mitre_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Villa_Lugano_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Villa_Luro_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Villa_Ortuzar_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Villa_Pueyrredon_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Villa_Real_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Villa_Riachuelo_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Villa_Santa_Rita_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Villa_Soldati_tejido.zip', 
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/tejido-urbano/Villa_Urquiza_tejido.zip']


print('====================')
print('Teijdos')
print('====================')
for i, link in enumerate(tejidos):
    print(f"Downloading zip {i+1}/{len(links)}...")
    neighbourhood = '_'.join(link.split('/')[-1].split('_')[:-1])
    urllib.request.urlretrieve(link, f"downloads/{neighbourhood}.zip")

In [0]:
caba = [
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/calles/callejero.csv',
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/manzanas/mapa-manzanas.csv',
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/secciones/secciones-catastrales.csv',
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/barrios/barrios.csv',
'https://cdn.buenosaires.gob.ar/datosabiertos/datasets/vivienda-durable-calidad-constructiva/hacinamiento-personas-por-cuarto-por-comuna.csv',
'http://cdn.buenosaires.gob.ar/datosabiertos/datasets/comunas/comunas.csv'
]

print('====================')
print('CABA')
print('====================')
for i, link in enumerate(caba):
    print(f"Downloading zip {i+1}/{len(caba)}...")
    name = link.split('/')[-2]
    urllib.request.urlretrieve(link, f"{name}.csv")

#This file would override barrios.csv with logic above.
urllib.request.urlretrieve('http://cdn.buenosaires.gob.ar/datosabiertos/datasets/barrios/barrios_comunas_p_Ciencia_de_Datos_y_PP.csv', 'barrio-por-comuna.csv')

# This is a heavy file. It will take a little bit if you want it. Uncomment next line.
# urllib.request.urlretrieve('http://cdn.buenosaires.gob.ar/datosabiertos/datasets/veredas/veredas-2019.csv', 'veredas.csv')

In [0]:
def unzip_path(path):
    files = os.listdir(path)
    for file in files:
        if file.endswith('.zip'):
            filePath = path+'/'+file
            zip_file = zipfile.ZipFile(filePath)
            for names in zip_file.namelist():
                zip_file.extract(names, f"{path}/{file.replace('.zip', '')}")
            zip_file.close()
    print(f'Succesfully unzipped all files in {path}')

unzip_path('downloads')

In [0]:
!mv downloads/Monserrat downloads/Montserrat
!mv downloads/Nunez/$(ls downloads/Nunez | grep dbf) downloads/Nunez/Nunez.dbf
!mv downloads/Villa_Del_Parque downloads/Villa_del_Parque

In [0]:
def load_df(path):
    dirs = dirs = [nhood for nhood in os.listdir(path) if os.path.isdir(f"{path}/{nhood}")]
    ret = ''
    for i, nhood in enumerate(dirs):
        if not os.path.isfile(f"{path}/{nhood}"):
            sf_path = f'{path}/{nhood}/{nhood}.shp'
            print(f'Loading {i+1}/{len(dirs)}...')
            sf = shapefile.Reader(sf_path, encoding='windows-1252')

            fields = [x[0] for x in sf.fields][1:]
            records = [y[:] for y in sf.records()]
            shps = [s.points for s in sf.shapes()]

            
            if len(ret) == 0:
                ret = pd.DataFrame(columns=fields, data=records)
                ret['BARRIO'] = nhood
            else:
                sd_df = pd.DataFrame(columns=fields, data=records)
                sd_df['BARRIO'] = nhood
                ret = pd.concat([ret, sd_df])
    return ret

df = load_df('downloads')
df[['seccion', 'manzana', 'parcela']] = df.SMP.str.split('-', expand=True)
df = df.drop(['ALTMETROS', 'SMP'], axis=1)

In [0]:
# Altura por seccion
s_x_alt = df[['ALTOS', 'seccion']].groupby(['seccion']).mean().reset_index()
s_x_alt['seccion'] = sec_x_alt['seccion'].str.lstrip("0").str.zfill(2)
s_x_alt = s_x_alt[s_x_alt['seccion'] != '']
secciones = geopandas.read_file('secciones.csv')
secciones['seccion'] = secciones['SECCION_']
secciones = secciones.drop('SECCION_', axis=1)

fig, ax = plt.subplots(1, 1)
secciones.merge(s_x_alt, on='seccion').plot(column='ALTOS', cmap='cool', ax=ax, legend=True)
ax.axis('off')
ax.set_title("Cantidad de pisos promedio por seccion")

plt.show()

In [0]:
# Altura por barrio
b_x_alt = df[['ALTOS', 'BARRIO']].groupby(['BARRIO']).mean().reset_index()
b_x_alt['barrio'] = b_x_alt['BARRIO'].str.upper().str.replace('_', ' ')
b_x_alt = b_x_alt.drop('BARRIO', axis=1)
barrios = geopandas.read_file('barrios.csv')
###### Normalize names between files ######
barrios.at[barrios[barrios['barrio'] == 'NUÑEZ'].index[0], 'barrio'] = 'NUNEZ'
barrios.at[barrios[barrios['barrio'] == 'MONSERRAT'].index[0], 'barrio'] = 'MONTSERRAT'
barrios.at[barrios[barrios['barrio'] == 'VILLA GRAL. MITRE'].index[0], 'barrio'] = 'VILLA GRAL MITRE'
###### Normalize names between files ######
fig, ax = plt.subplots(1, 1)
barrios.merge(b_x_alt, on='barrio').plot(column='ALTOS', cmap='cool',ax=ax, legend=True)
ax.axis('off')
ax.set_title("Cantidad de pisos promedio por barrio")

plt.show()

In [0]:
manzanas = geopandas.read_file('manzanas.csv')
barrios = list(set(df['BARRIO']))
for pointer in range(0, len(barrios), 4):
  fig, axes = plt.subplots(2, 2)
  axes = [item for sublist in axes for item in sublist]
  for j, ax in enumerate(axes):
    barrio = barrios[pointer+j]
    print(f'Plotting {barrio}...')
    m_x_alt = df[df['BARRIO'] == barrio]
    m_x_alt = m_x_alt[['ALTOS', 'seccion', 'manzana']].groupby(['seccion', 'manzana']).mean().reset_index()
    m_x_alt['sm'] = m_x_alt[['seccion', 'manzana']].agg('-'.join, axis=1)

    merged = manzanas.merge(m_x_alt, on='sm', how='inner')
    merged.plot(column='ALTOS', cmap='cool', ax=ax, legend=True)
    ax.axis('off')
    ax.set_title(f"{barrio.replace('_', ' ')}")

fig.suptitle('Cantidad de pisos promedio por manzana', fontsize=16)
plt.show()